## Analysis of Faculty Data
### Richard M. Murray, 18 Jan 2021

This notebook creates a "master spreadsheet" of all faculty teaching and service activities for the last five years.  The data are compiled from a set of spreadsheets that have been collecting from different locations are are parsed into data entries via custom parsing routes infor each spreadsheet.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import re
import bbedata as bbe

# Create a dataframe to store the result
bbe_df = pd.DataFrame(columns=["Name", "Committee", "Year"])

### List of BBE faculty
Generate a list of all BBE faculty and the type of appointment they have.

In [ ]:
# Create a set to hold the names of BBE faculty (for searching)
bbe_faculty = bbe.load_faculty_data("BBE Faculty.xlsx")
bbe_faculty.info()
bbe_names = bbe_faculty.Name.unique()

In [ ]:
bbe_faculty.tail(20)

In [ ]:
# Create and initialize a dataframe for holding all of the results
fac_df = pd.DataFrame(columns=['Name', 'Activity', 'Role', 'AY', 'Level', 'Type', 'Size', 'Hrs/Yr'])

# Add a placehold for every faculty member (to allowing finding people not serving on committees)
for index, faculty in bbe_faculty.iterrows():
    fac_df = fac_df.append(
        {'Name': faculty['Name'],
         'Activity': "Research",
         'Role': "PI",
         'AY': "2020-21",
        },
    ignore_index=True)

### Institute committee membership

In [ ]:
# Read data from committee membership
committees = pd.read_excel("BBE Committee Membership Report 2001-2021.xlsx")
committees = committees.astype({'Term Start': np.datetime64, 'Term End': np.datetime64})

# Go through each year that we are interested in
for year in range(2014, 2021):
    bbe_data = committees[
        (committees["Term Start"] < np.datetime64(str(year))) & 
        (np.datetime64(str(year)) < committees["Term End"])
    ][["Last Name", "First Name", "Committee", "Title", "Special Role"]]
    
    # Store the data in a new dataframe
    for index, entry in bbe_data.iterrows():
        # Construct the person's name
        name = bbe.create_name(entry["Last Name"], entry["First Name"])
        
        # See if the person is in BBE
        if name not in bbe_names:
            continue
            
        # Figure out the role
        role = entry["Title"]
        if entry["Special Role"] is not np.nan and role is not np.nan:
            role += ", " + entry["Special Role"]
        elif role is np.nan:
            role = entry["Special Role"]
            
        # Figure out the type of activity
        if entry["Title"] == "Chair" or entry["Title"] == "Vice-Chair" or entry["Title"] == "Secretary":
            type = "Leadership"
        else:
            type = "Service"
        
        # Add the person to the data frame
        fac_df = fac_df.append(
            {"Name": name,
             'Activity': entry["Committee"],
             'Role': role,
             'AY': "%d-%d" % (year, year+1-2000),
             'Level': "Institute",
             'Type': type
            },
        ignore_index=True)
fac_df.info()

### Teaching

In [ ]:
# Read data from teaching assignments
courses = bbe.read_regis_data("All courses past 5 years 210120.xlsx")
courses.info()

# Go through each course and create an entry for that instructor
for index, course in courses.iterrows():
    # Skip research courses
    if course['Type'] == "Research":
        continue
    
    # Add an entry for each instructor
    instructors = course['Instructors'].split('; ')
    for name in instructors:
        # Make sure this is a BBE faculty member
        if name not in bbe_names:
            continue
            
        # Add the person to the data frame
        fac_df = fac_df.append(
            {"Name": name,
             'Activity': "%s - %s" % (course['Course'], course['Title']),
             'Role': 'Instructor' if len(instructors) == 1 else 'Co-instructor',
             'AY': course['AY'],
             'Level': "Option",
             'Type': "Teaching",
             'Size': course['Enrollment'],
            },
        ignore_index=True)

fac_df.tail()

### Division committees

In [ ]:
# Set up dictionary with file names
div_committee_files = {
    '2015-16': "BBE Division Committee List 11-20-15.xlsx",
    '2016-17': "BBE DIVCommittees 2016-2017 updated 3-29-17 FINAL.xlsx",
    '2017-18': "BBE DIVCommittees 2017-2018 updated 3-26-18 FINAL.xlsx",
    '2018-19': "BBE DIVCommittees 2018-2019 FINAL.xlsx",
    '2019-20': "BBE DIVCommittees 2019-2020 rev.1.xlsx",
    '2020-21': "BBE DIVCommittees 2020-2021 draft 4 _1-14-21.xlsx"
}

for year, filename in div_committee_files.items():
    print("Reading: ", filename)
    div_committees = pd.read_excel(div_committee_files[year])
    div_committees.info()

    # Patterns for parsing the faculty member name
    pattern_name_only = re.compile(r"^\s*([a-zA-Z \-]*, \S*)\s*$")
    pattern_first_last_only = re.compile(r"^\s*(\S*\s*[a-zA-Z \-]*)\s*$")
    pattern_exec_officer = re.compile(
        r"^\s*(\S*)\s*-\s*(\S*, \S*)\s*\([0-9]*/[0-9]*/[0-9]*-[0-9]*/[0-9]*/[0-9]*\)\s*$")
    pattern_name_date_range = re.compile(
        r"^(.*, \S*) \([0-9]*/[0-9]*/[0-9]*-[0-9]*/[0-9]*/[0-9]*\)$")
    pattern_name_parens = re.compile(r"^(.*, \S*)\s*\((.*)\)")
    pattern_first_last_parens = re.compile(r"^(\S*\s*[a-zA-Z \-]*)\s*\((.*)\)$")
    pattern_name_dash = re.compile(r"^(.*, \S*) *- *(\S*)$")

    for row, data in div_committees.iterrows():
        # Get the data from the spreadsheet
        faculty_member = data["Faculty Member"]
        committee_position = data["Committee/Position"]
    
        # Skip empty rows
        if faculty_member is np.nan:
            continue
        
        # Set the defaults
        level = "Division"
        role = np.nan
    
        # Parse the committee and remember it
        if committee_position is not np.nan:
            activity = committee_position
    
        #
        # Parse the name of the faculty member
        #
        name = np.nan
    
        # Named executive officer
        match = pattern_exec_officer.match(faculty_member)
        if match is not None:
            name = match.group(2)
            
            # Executive officer or division chair
            if committee_position == "Chair":
                activity = "BBE Division"
                role = "Division Chair"
                type_ = "Leadership"
            elif committee_position is np.nan or \
                committee_position.startswith("Executive Officer"):
                activity = "Executive Officer (" + match.group(1) + ")"
                role = "Executive Officer"
                type_ = "Leadership"
            else:
                raise ValueError("Coudn't parse position '", committee_position, "'")
        
        # Committee chair or other role
        match = pattern_name_parens.match(faculty_member)
        if match is None:
            match = pattern_name_dash.match(faculty_member)
        if match is None:
            match = pattern_first_last_parens.match(faculty_member)
        if match is not None and name is np.nan:
            name = match.group(1)
            type_ = np.nan
            if match.group(2) == "Chair":
                if div_committees.loc[row+1, "Committee/Position"] == \
                    "Graduate Student Admissions Committee":
                    activity = "Graduate Student Admissions Committee"
                role = "Chair"
                type_ = "Leadership"
            elif activity.startswith("Training Grant"):
                activity = "Training Grant (%s)" % match.group(2)
                role = "Director"
                type_ = "Leadership"
            elif activity.startswith("Qualifying Exam"):
                activity = "Qualifying Examinations (%s)" % match.group(2)
                role = np.nan
                
            
        # Regular committee member
        match = pattern_name_only.match(faculty_member)
        if match is None:
            match = pattern_first_last_only.match(faculty_member)
        if match is not None and name is np.nan:
            name = match.group(1)
            type_ = "Service"
            
        # Make sure that we were able to parse the name
        if name is np.nan:
            print("Couldn't parse '", faculty_member, "'")
            continue
            
        # Make sure that we found a name that we recognize
        name_orig = name
        name = bbe.normalize_name(name)
        if name not in bbe_names:
            print("Non-BBE name:", name, "[%s]" % name_orig)
            continue

        # Add the person to the data frame
        fac_df = fac_df.append(
            {'Name': name,
             'Activity': activity,
             'Role': role,
             'AY': year,
             'Level': level,
             'Type': type_
            }, ignore_index=True)
        
fac_df.tail()

### Administrative committees

In [ ]:
pattern_name_only = re.compile(r"^(.*, \S*) .*")
pattern_name_parens = re.compile(r"^(.*, \S*) \((.*)\)$")
pattern_committee_parens = re.compile(r"^(.*) \((.*)\)$")

level = "Institute"

for year in range(2015, 2020):
    sheet = pd.read_excel("BBE_Administrative  & Faculty Committee Service_2015-2020_1.16.21.xlsx", str(year))
    
    for row, data in sheet.iterrows():
        # Look for the name
        if data["Name"] is np.nan:
            continue
        match = pattern_name_only.match(data["Name"])
        if match is None or match.group(1) not in bbe_names:
            continue
        name = bbe.normalize_name(match.group(1))
        
        # Parse the committee
        if data["Administrative Committees"] is np.nan:
            continue
        type_ = np.nan
        for committee in data["Administrative Committees"].split('; '):
            # See if the person had a specific role
            match = pattern_name_parens.match(committee)
            if match is None:
                match=pattern_committee_parens.match(committee)
            if match is None:
                activity = committee
                role = np.nan
                type_ = np.nan
            else:
                activity = match.group(1)
                role = match.group(2)
                type_ = "Leadership" if role == "Chair" else np.nan
                
            # Add the person to the data frame
            fac_df = fac_df.append(
                {'Name': name,
                 'Activity': activity,
                 'Role': role,
                 'AY': "%d-%d" % (year, year+1-2000),
                 'Level': level,
                 'Type': type_
                }, ignore_index=True)
        
fac_df.tail()

In [ ]:
# Write the data frame
fac_df.drop_duplicates(inplace=True)
fac_df.to_excel("BBE Faculty Activities.xlsx", "Activities")